# Debugging Tools using Quantum Simulators

For testing, we run the quantum programs in several ways such as using fake backends, reference primitives, Aer simulators and creating stabilizied circuits.

## Fake Backends

Using Fake Backends from "qiskit_ibm_runtime.fake_provider" module.

In [10]:
#Working with Sampler primitive only
from qiskit.circuit import QuantumCircuit
from qiskit import transpile
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

fake_manila = FakeManilaV2()

# Bell Circuit (|00> + |11>)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()
 
# transpilation
isa_qc = transpile(qc, backend = fake_manila, optimization_level =1)
 
# You can use a fixed seed to get fixed results.
options = {"simulator": {"seed_simulator": 42}}
sampler = Sampler(mode=fake_manila, options=options)
 
result = sampler.run([isa_qc]).result()
data = result[0].data
counts = data.meas.get_counts()
print(f"Counts: {counts}")

Counts: {'11': 457, '00': 503, '01': 27, '10': 37}


Theoretically, we should get zero for 01 and 10. Due to errors in quantum computing, we get nonzero values. This method thus also models noise.

## Reference Primitives

In [11]:
#using only Sampler primitive
from qiskit.circuit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
 
sampler = StatevectorSampler()
 
# Bell Circuit (|00> + |11>)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# execute circuit with Sampler
job = sampler.run([qc])
result = job.result()
data = result[0].data
counts = data.meas.get_counts()
print(f"Counts: {counts}")


Counts: {'00': 494, '11': 530}


This method does not support modelling device noise. So, we see the values of 01 and 10 states are exactly zero. 

## Qiskit Aer 

QiskitAer - A high-performance Qcircuit simulator that we can use in place of reference primitives for better performance and implementing device noise.

In [14]:
#Using Sampler primitive only
from qiskit import QuantumCircuit,transpile
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 

sampler = SamplerV2()

# Step 1: Create a simple circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# execute circuit with Sampler
job = sampler.run([qc])
result = job.result()
data = result[0].data
counts = data.meas.get_counts()
print(f"Counts: {counts}")

Counts: {'00': 536, '11': 488}


In above code, there is no noise in the system. One can implement noise models as well using this QiskiAer module.

In [22]:
from qiskit import QuantumCircuit,transpile
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit_aer.noise import NoiseModel,depolarizing_error

#Noise Model
noise_model = NoiseModel()
cx_depolarizing_prob = 0.02
noise_model.add_all_qubit_quantum_error(
    depolarizing_error(cx_depolarizing_prob, 2), ["cx"]
)

noisy_sampler = Sampler(options=dict(backend_options=dict(noise_model=noise_model)))

# Create a simple circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# execute circuit with Sampler
job = noisy_sampler.run([qc])
result = job.result()
data = result[0].data
counts = data.meas.get_counts()
print(f"Counts: {counts}")

Counts: {'11': 510, '00': 506, '10': 6, '01': 2}


After incorporting Noise model, we get nonzero values for 01 and 10 states.

## Stabilizer Circuits

Stabilizer or Clifford circuits, are an important restricted class of quantum circuits that can be efficiently simulated classically.